<a href="https://colab.research.google.com/github/tomonari-masada/course2024-stats1/blob/main/normalgamma_Rosenthal_and_Jacobson_1968.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rosenthal and Jacobson (1968) の実験の結果を分析

* データは、原論文のものではなく、それに似せて作ったもの。
 * [STA 360/602: Bayesian Methods and Modern Statistics @ Duke University](http://www2.stat.duke.edu/~rcs46/bayes17.html)の[Module 4](http://www2.stat.duke.edu/~rcs46/modern_bayes17/lecturesModernBayes17/lecture-4/04-normal-gamma.pdf)より拝借したデータ。


> Do a teacher’s expectations influence student achievement? In a
famous study, Rosenthal and Jacobson (1968) performed an
experiment in a California elementary school to try to answer this
question. At the beginning of the year, all students were given an
IQ test. For each class, the researchers randomly selected around
20% of the students, and told the teacher that these students were
“spurters” that could be expected to perform particularly well that
year. (This was not based on the test—the spurters were randomly
chosen.) At the end of the year, all students were given another IQ
test.



## 問題設定
* 教師が期待をかけるか否かで学生の学修に影響があるかを知りたい。
* このことを、$P(\mu_s > \mu_c | \mathbf{x}_s, \mathbf{x}_c)$を調べることで、明らかにする。
 * $\mathbf{x}_s$: spurters（期待をかけられた学生たち）のIQ変化量データ群
 * $\mathbf{x}_c$: controls（その他の学生たち）のIQ変化量データ群
 * $\mu_s$: spurtersのIQ変化量の平均値
 * $\mu_c$: controlsのIQ変化量の平均値

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## モデリングの方針
* IQスコア変化量は、正規分布に従うと仮定する。
* 事前分布としては、正規ガンマ分布を用いる。

## データ例
* spurtersとcontrolsのIQスコアの変化量

In [ ]:
x_s = [18, 40, 15, 17, 20, 44, 38]
x_c = [-4, 0, -19, 24, 19, 10, 5, 10,
       29, 13, -9, -8, 20, -1, 12, 21,
       -7, 14, 13, 20, 11, 16, 15, 27,
       23, 36, -33, 34, 13, 11, -19, 21,
       6, 25, 30,22, -28, 15, 26, -1, -2,
       43, 23, 22, 25, 16, 10, 29]

In [ ]:
x_s = np.array(x_s)
x_c = np.array(x_c)
N_s = len(x_s)
N_c = len(x_c)

In [ ]:
ax = sns.histplot({'spurters':x_s, 'controls':x_c}, binwidth=2)
ax.set_title('data distribution');

## 参考: Welchのt検定
* https://bellcurve.jp/statistics/course/9936.html
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind
  * この`scipy.stats.ttest_ind`では、df（自由度）は以下のように計算している。
  * `df = (vn1 + vn2) ** 2 / (vn1**2 / (n1 - 1) + vn2**2 / (n2 - 1))`
  * ただし `vn1 = v1 / n1`、`vn2 = v2 / n2`


In [ ]:
from scipy import stats

t_score = stats.ttest_ind(x_s, x_c, equal_var=False, alternative='greater')
print(t_score)

## 事前分布のパラメータの設定

* $\alpha$は、事前分布がデータ1個分の情報を持つように設定されている。
 * 事後分布のshapeパラメータが$\alpha + N / 2$となるため。
* $\beta$は、IQスコアの変化量の標準偏差$\sqrt{\beta / \alpha}$が10ぐらいになるように設定されている。
 * ガンマ分布$\mbox{Gamma}(\alpha, \beta)$に従う確率変数の値の平均は$\alpha / \beta$。
 * ガンマ分布は、ここでは、IQスコアの変化量が従う正規分布の精度（＝分散の逆数）が従う分布として使われている。

In [ ]:
alpha = 0.5
beta = 100 * alpha

* $\lambda_0$は、どう設定していいか分からないので$1$にする。
* $\mu_0$は、spurtersとcontrolsを合わせたデータ全体の平均値に設定する。

In [ ]:
lambda_0 = 1.0
mu_0 = (x_s.sum() + x_c.sum()) / (N_s + N_c)
print(mu_0)

## 事後分布のパラメータの計算
* データ集合が与えられれば、この計算は実行できる。

In [ ]:
mu_s = (lambda_0 * mu_0 + sum(x_s)) / (lambda_0 + N_s)
mu_c = (lambda_0 * mu_0 + sum(x_c)) / (lambda_0 + N_c)
lambda_s = lambda_0 + N_s
lambda_c = lambda_0 + N_c
alpha_s = alpha + N_s / 2
alpha_c = alpha + N_c / 2
beta_s = beta + (((x_s - x_s.mean()) ** 2).sum() + lambda_0 * N_s * (x_s.mean() - mu_0) ** 2 / (lambda_0 + N_s)) / 2
beta_c = beta + (((x_c - x_c.mean()) ** 2).sum() + lambda_0 * N_c * (x_c.mean() - mu_0) ** 2 / (lambda_0 + N_c)) / 2

## 事後分布からサンプリング
* 正規ガンマ分布$\mu, \tau \sim \mbox{NormalGamma}(\mu_0, \lambda_0, \alpha, \beta)$からサンプリングするには・・・
* まず、ガンマ分布$\tau \sim \mbox{Gamma}(\alpha, \beta)$から$\tau$をサンプリングし、
* 次に、正規分布$\mu \sim N(\mu_0, 1 / (\lambda_0\tau))$から$\mu$をサンプリングすればよい。

In [ ]:
from scipy.stats import gamma

In [ ]:
rv_s = gamma(alpha_s, scale=1/beta_s)
rv_c = gamma(alpha_c, scale=1/beta_c)

In [ ]:
n_samples = 1000

tau_s_samples = rv_s.rvs(n_samples)
mean_s_samples = np.random.randn(n_samples) / np.sqrt(lambda_s * tau_s_samples) + mu_s
tau_c_samples = rv_c.rvs(n_samples)
mean_c_samples = np.random.randn(n_samples) / np.sqrt(lambda_c * tau_c_samples) + mu_c

* 平均パラメータの分布のヒストグラムを描く
 * このヒストグラムが、平均パラメータが従う事後分布を近似しているはず。

In [ ]:
ax = sns.histplot({'spurters':mean_s_samples, 'controls':mean_c_samples})
ax.set_title('posterior distribution of mean parameter');

* 標準偏差パラメータの分布のヒストグラムを描く

In [ ]:
sigma_s_samples = np.sqrt(1 / tau_s_samples)
sigma_c_samples = np.sqrt(1 / tau_c_samples)
ax = sns.histplot({'spurters':sigma_s_samples, 'controls':sigma_c_samples})
ax.set_title('posterior distribution of stdev parameter');

## $\mu_s > \mu_c$となる確率の計算

In [ ]:
(mean_s_samples > mean_c_samples).sum() / n_samples

## 予測分布を求める
* t分布について、自由度とlocationとscaleを、 指定する。

In [ ]:
from scipy.stats import t

In [ ]:
n_samples = 10000

rv_s = t(2 * alpha_s, loc=mu_s, scale=np.sqrt(beta_s * (lambda_s + 1) / (alpha_s * lambda_s)))
x_s_samples = rv_s.rvs(n_samples)
rv_c = t(2 * alpha_c, loc=mu_c, scale=np.sqrt(beta_c * (lambda_c + 1) / (alpha_c * lambda_c)))
x_c_samples = rv_c.rvs(n_samples)

In [ ]:
sns.histplot({'spurters':x_s_samples, 'controls':x_c_samples});

* 同じ仮定を置ける状況下で、次に観測されるspurterのIQ変化量が、次に観測されるcontrolのIQ変化量よりも大きい確率
  * 平均パラメータではなく、予測値を比べている点に注意。

In [ ]:
(x_s_samples > x_c_samples).sum() / n_samples

## モデル選択
* log pointwise predictive densityによってモデルを選択する。

### ここまでの復習

* ここまで使っていたハイパーパラメータ

In [ ]:
alpha = 0.5
beta = 100 * alpha
lambda_0 = 1.0
mu_0 = (x_s.sum() + x_c.sum()) / (len(x_s) + len(x_c))

* 事後分布のパラメータを求めるヘルパ関数

In [ ]:
def posterior_parameters(x):
  N = len(x)
  mean_x = x.mean()
  mu_ = (lambda_0 * mu_0 + sum(x)) / (lambda_0 + N)
  lambda_ = lambda_0 + N
  alpha_ = alpha + N / 2
  beta_ = (
      beta + (((x - mean_x) ** 2).sum()
      + lambda_0 * N * (mean_x - mu_0) ** 2 / (lambda_0 + N)) / 2
  )
  return mu_, lambda_, alpha_, beta_

* $\mu_s > \mu_c$となる確率の計算

In [ ]:
n_samples = 100000

mean_samples = []
for x in [x_s, x_c]:
  mu_, lambda_, alpha_, beta_ = posterior_parameters(x)
  rv = gamma(alpha_, scale=1/beta_)
  tau_samples = rv.rvs(n_samples)
  mean_samples.append(np.random.randn(n_samples) / np.sqrt(lambda_ * tau_samples) + mu_)

prob = (mean_samples[0] > mean_samples[1]).sum() / n_samples
print(f"P(mu_s > mu_c | x_s, x_c) = {prob:.3f}")

### log pointwise predictive density
* データセットから1点除外する。
* 除外したもの以外全てを使って事後分布を求める。
* その事後分布のもとで、除外した1点の対数予測確率を求める。

In [ ]:
def logpdf(x, mu_, lambda_, alpha_, beta_):
  rv = t(2 * alpha_, loc=mu_, scale=np.sqrt(beta_ * (lambda_ + 1) / (alpha_ * lambda_)))
  return rv.logpdf(x)

In [ ]:
def log_pointwise_predictive_density(x):
  temp_logpdf = []
  for idx in range(len(x)):
    mu_, lambda_, alpha_, beta_ = posterior_parameters(np.delete(x, idx))
    temp_logpdf.append(logpdf(x[idx], mu_, lambda_, alpha_, beta_))
  return np.array(temp_logpdf).mean()

### ハイパーパラメータの設定 (1)

In [ ]:
alpha = 0.5
beta = 100 * alpha
lambda_0 = 1.0
mu_0 = (x_s.sum() + x_c.sum()) / (len(x_s) + len(x_c))

In [ ]:
lppds = []
for x_ in [x_s, x_c]:
  lppds.append(log_pointwise_predictive_density(x_))
print(f"{np.array(lppds).mean():.3f} (x_s: {lppds[0]:.3f}, x_c: {lppds[1]:.3f})")

### ハイパーパラメータの設定 (2)

In [ ]:
alpha = 0.5
beta = 100 * alpha
lambda_0 = 1.0
mu_0 = x_c.sum() / len(x_c) # mu_0をcontrolsだけで決める

In [ ]:
lppds = []
for x_ in [x_s, x_c]:
  lppds.append(log_pointwise_predictive_density(x_))
print(f"{np.array(lppds).mean():.3f} (x_s: {lppds[0]:.3f}, x_c: {lppds[1]:.3f})")

### ハイパーパラメータの設定 (3)

In [ ]:
alpha = 5.0 # 事前分布を強くする
beta = 100 * alpha
lambda_0 = 1.0
mu_0 = (x_s.sum() + x_c.sum()) / (len(x_s) + len(x_c))

In [ ]:
lppds = []
for x_ in [x_s, x_c]:
  lppds.append(log_pointwise_predictive_density(x_))
print(f"{np.array(lppds).mean():.3f} (x_s: {lppds[0]:.3f}, x_c: {lppds[1]:.3f})")

* alphaを探索してみる？

In [ ]:
lambda_0 = 1.0
mu_0 = (x_s.sum() + x_c.sum()) / (len(x_s) + len(x_c))

for alpha in (np.arange(30) + 1) * 0.5:
  beta = 100 * alpha
  lppds = []
  for x_ in [x_s, x_c]:
    lppds.append(log_pointwise_predictive_density(x_))
  print(f"alpha={alpha} | {np.array(lppds).mean():.5f} (x_s: {lppds[0]:.3f}, x_c: {lppds[1]:.3f})")